## Importation des fichiers utiles

In [2]:
import urllib.request
import json
import pandas as pd
import numpy as np
from PIL import Image, ExifTags
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from sklearn.linear_model import Perceptron

NOMBRE_IMAGE=141

<br>

## Demande de Json
Nous Faisons une Requète WikiData pour demander un Json contenant le lien menant à des libres d'une certaine catégorie. Nous avons essayé plusieurs types d'images différentes :

mythical creature = Q2239243<br>
photograph = Q125191<br>
flower = Q506<br>
fruit = Q1364<br>
Chien = Q144<br>
lac  = Q23397<br>


SELECT ?image ?im WHERE {
  ?image wdt:P31 wd:Q506.
  ?image wdt:P18 ?im
  }
LIMIT 200

In [3]:
lstSources = {"Fruit":1364,"Dog":144,"Lake":23397}
limitPerSources = 5
sources = {}
tagTemp = []
for key,value in lstSources.items():    #Get all images to download
    url ="https://query.wikidata.org/sparql?query=SELECT%20%3Fimage%20%3Fim%20WHERE%20%7B%0A%20%20%3Fimage%20wdt%3AP31%20wd%3AQ"+str(value)+".%0A%20%20%3Fimage%20wdt%3AP18%20%3Fim%0A%20%20%7D%0ALIMIT%20"+str(limitPerSources)+"&format=json"
    response = urllib.request.urlopen(url)
    myJson = json.loads(response.read().decode("utf-8"))
    
    for image in myJson["results"]["bindings"]:  #Add the tag to the image
        image['tag'] = key
        
    if sources == {}:   #Make one big json containing all images
        sources = myJson
    else:
        sources["results"]["bindings"] = sources["results"]["bindings"]+myJson["results"]["bindings"]

myJsonNormalized = pd.json_normalize(sources)

<br>

## Télechargement des images ##
A partir du Json contenant les liens menant aux images, nous lancons des requètes URLLIB pour télecharger ces images.

In [4]:
#telechargement des images

responseJson = myJsonNormalized["results.bindings"]
index=0
lstTag=[]
for i in range(len(responseJson[0])):   
    imageLink = responseJson[0][i]["im"]["value"]
    lstTag.append(responseJson[0][i]["tag"])
    print(index)
    
    im = "./images/image"+str(index)+".jpg"
    urllib.request.urlretrieve(imageLink, im)
    index +=1

    try:
        imgfile = Image.open(im).convert("RGB")
    except:
        index-=1
        del lstTag[-1]
        print("pas fonctionne")
    
NOMBRE_IMAGE=index

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


HTTPError: HTTP Error 429: Too many requests. Please comply with the User-Agent policy to get a higher rate limit: https://meta.wikimedia.org/wiki/User-Agent_policy

<br>

## Récupération des métadonnées
L'obhectif de cette partie est de récupérer les informations des images, nous récupérons les images suivantes :<br>
1. Récupération Exif utiles
2. Récupération couleurs dominantes (K-means)
3. Récupération taille des images
4. Détermination portrait ou paysage

Apres cela, nous avons creer un Json **metaDataJson.json** contenant toutes les informationns precedentes

In [5]:
# Récupération des métadonnées

metaData ={}
for i in range(NOMBRE_IMAGE):
    im = "./images/image"+str(i)+".jpg"
    #try:
    imgfile = Image.open(im).convert("RGB")

    #Recupération Exif utiles
    exif = {}
    exifCritere= ("Model","Make","ImageDescription","ExposureTime","GPSInfo","ISO","DateTimeOriginal")
    for k,v in  imgfile.getexif().items() :
        if k in ExifTags.TAGS :
            if ExifTags.TAGS[k] in exifCritere:
                exif = {ExifTags.TAGS[k]:v}

    # Recupération couleurs dominantes (K-means)
    numarray = np.array(imgfile.getdata(), np.uint8)
    clusters  = MiniBatchKMeans(n_clusters=3, n_init=2) 
    clusters.fit(numarray)
    L = clusters.cluster_centers_.astype(int).tolist()
    if L[0][0]>L[0][1] and L[0][0]>L[0][2]:
        domColor = "Red"
    elif L[0][1]>L[0][0] and L[0][1]>L[0][2]:
        domColor = "Green"
    elif L[0][2]>L[0][0] and L[0][2]>L[0][1]:
        domColor = "Blue"
    elif L[0][0]==L[0][1]==L[0][2]:
        domColor = "White"
    else:
        domColor = "None"

    # Recupération taille des images
    width = imgfile.width
    height = imgfile.height
    mode = imgfile.mode

    # Détermination portrait ou paysage
    if abs(1-width/height)<0.1 :
        orientation='carre' 
    elif width>height:
        orientation='paysage'
    else: 
        orientation="portrait"

    # Get image size (big, small, ...)
    pixelCount = width * height

    if pixelCount >= 1920*1080:
        size = "Grande"
    elif pixelCount >= 1280*720:
        size = "Moyenne"
    elif pixelCount >= 720*480:
        size = "Petite"
    elif pixelCount < 720*480:
        size = "Vignette"
    else:
        size = "WTF"

    metaData["image"+str(i)] = { "width" : width, "height": height, "exif": exif, "mode":mode, "tags":[lstTag[i]], "couleurs":L, "orientation":orientation, "size":size, "dom":domColor}
    #except Exception:
    print("l'image "+str(i) +" ne peut pas etre ouverte")
metaData

metaDatajson = json.dumps(metaData)
f = open("metaDataJson.json", "w")
f.write(metaDatajson)
f.close()



c:\Users\gabin\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:887: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 4096 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


l'image 0 ne peut pas etre ouverte


c:\Users\gabin\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:887: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 4096 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


l'image 1 ne peut pas etre ouverte


c:\Users\gabin\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:887: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 4096 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


...

l'image 12 ne peut pas etre ouverte


c:\Users\gabin\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:887: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 4096 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


l'image 13 ne peut pas etre ouverte


c:\Users\gabin\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:887: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 4096 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(


l'image 14 ne peut pas etre ouverte


## Analyse

Consiste en 2 parties, l'analyse couleur, pour savoir si l'image proposée à des couleurs proche des images aimées par l'utilisateur. Et l'analyse globale qui utilise un arbre de décision et les paramètres de metadataJson pour décider si l'image peut etre proposée ou non à l'utilisateur.

In [65]:
def flatten(l):
    return [item for sublist in l for item in sublist]


def analyseCouleur(numUser,numImage): 
    jsonUser=json.load(open("jsonUser.json"))[numUser]
    jsonMeta=json.load(open("metaDatajson.json"))
    couleurs=[]

    for image in jsonMeta:
        if image in jsonUser["data"]["image"]:
            couleurs.append(flatten(jsonMeta[image]["couleurs"]))
            
    numarray=np.array(couleurs)
    like=np.array(jsonUser["data"]["resultat"])
    
    # modele des couleurs
    percep = Perceptron(max_iter=1000)
    percep.fit(numarray, like)
    
    # prediction
    nomImage="image"+str(numImage)
    x_predict = np.array([flatten(jsonMeta[image]["couleurs"])])
    y_predict = percep.predict(x_predict)

    return y_predict[0]
analyseCouleur(1,1) 

KeyError: 1

In [7]:

#createUser("myName", "effef")


['NotFavorite']
[0.50943396 0.18867925 0.30188679 0.        ]


## Systeme de recommendation

In [3]:
def createUser(nom, prenom, images,like):
    jsonUser=json.load(open("jsonUser.json"))
    
    userId="user"+str(len(jsonUser))
    print(userId)
    jsonUser[userId]={"nom" : nom,"prenom" : prenom, "images":[],"result":[],"year":"","size":"","tagFav":"","orientation":"","couleursFav":""
        
    }
    
    jsonUserStr = json.dumps(jsonUser, indent=4)
    f = open("jsonUser.json", "w")
    f.write(jsonUserStr)
    f.close()  
createUser("myName", "effef",1,1)


user4


In [ ]:
from sklearn import tree
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Get image data
jsonMetaDF = pd.read_json("metaDataJson.json")
lstTags = jsonMetaDF.loc["tags"].to_list()
lstOrientation = jsonMetaDF.loc["orientation"].to_list()

# Get user like
userDF = pd.read_json("jsonUser.json")
userIndex = 0 # Get user0
userImages = userDF.loc["images"][userIndex]
userLike = userDF.loc["result"][userIndex]

data = []
for item in userImages:
    imageDesc = []
    #imageDesc.append(jsonMetaDF.loc["couleurs"][item])
    imageDesc.append(jsonMetaDF.loc["dom"][item])
    imageDesc.append(jsonMetaDF.loc["tags"][item][0])
    imageDesc.append(jsonMetaDF.loc["size"][item])
    imageDesc.append(jsonMetaDF.loc["orientation"][item])
    data.append(imageDesc)


result = [None]*len(userImages)
for item in jsonMetaDF:
    if item in userImages:
        ind = userImages.index(item)
        if userLike[ind] == 1:
            result[ind]="Favorite"
        else :
            result[ind]="NotFavorite"

# creating dataframes
dataframe = pd.DataFrame(data, columns=["dom", "tags", "size", "orientation"])
resultframe = pd.DataFrame(result, columns=["favorite"])

# generating numerical labels
le1 = LabelEncoder()
dataframe["dom"] = le1.fit_transform(dataframe["dom"])

le2 = LabelEncoder()
dataframe["tags"] = le2.fit_transform(dataframe["tags"])

le3 = LabelEncoder()
dataframe["size"] = le3.fit_transform(dataframe["size"])

le4 = LabelEncoder()
dataframe["orientation"] = le4.fit_transform(dataframe["orientation"])

le5 = LabelEncoder()
resultframe["favorite"] = le5.fit_transform(resultframe["favorite"])

# Use of decision tree classifiers
dtc = tree.DecisionTreeClassifier()
dtc = dtc.fit(dataframe.values, resultframe)

print(dtc.feature_importances_)
# prediction
prediction = dtc.predict(
    [
        [
            le1.transform(["Blue"])[0],
            le2.transform(["Dog"])[0],
            le3.transform(["Grande"])[0],
            le4.transform(["portrait"])[0],
        ]
    ]
)
print(le5.inverse_transform(prediction))
print(dtc.feature_importances_)

In [6]:
def cyclePropositionImage(numUser):
    jsonMetaDF = pd.read_json("metaDataJson.json")
    
cyclePropositionImage(1)
jsonMetaDF






,image0,image1,image2,image3,image4,image5,image6,image7,image8,image9,image10,image11,image12,image13,image14
width,2277,2560,800,2048,743,1329,1173,2180,2649,391,1072,1408,3456,2816,3264
height,1832,1920,600,1536,624,1201,1496,1617,3749,295,804,1056,2592,2112,2448
exif,{},{'Model': 'DMC-FZ20'},{'Model': 'E4500'},{'Model': 'hp photosmart 735'},{},{},{},{'Model': 'DMC-FZ20'},{'Model': 'Canon EOS R'},{},{'Model': 'KODAK EasyShare Z981 Digital Camera...,{},{'Model': 'DSC-HX1'},"{'Model': 'FE-140,X-725'}",{'Model': 'Canon PowerShot S80'}
mode,RGB,RGB,RGB,RGB,RGB,RGB,RGB,RGB,RGB,RGB,RGB,RGB,RGB,RGB,RGB
tags,[Fruit],[Fruit],[Fruit],[Fruit],[Fruit],[Dog],[Dog],[Dog],[Dog],[Dog],[Lake],[Lake],[Lake],[Lake],[Lake]
couleurs,"[[131, 118, 106], [26, 23, 57], [216, 212, 173]]","[[234, 118, 151], [101, 19, 31], [188, 66, 86]]","[[183, 114, 9], [236, 234, 228], [196, 175, 142]]","[[76, 48, 21], [178, 153, 121], [112, 92, 63]]","[[216, 179, 104], [4, 57, 148], [142, 96, 36]]","[[72, 72, 72], [227, 227, 227], [149, 149, 149]]","[[140, 140, 140], [186, 186, 186], [81, 81, 81]]","[[155, 141, 124], [73, 66, 53], [191, 183, 170]]","[[73, 72, 75], [174, 137, 132], [174, 181, 195]]","[[156, 137, 119], [78, 95, 40], [222, 229, 223]]","[[120, 140, 121], [206, 208, 200], [83, 95, 66]]","[[105, 124, 138], [206, 221, 233], [31, 41, 45]]","[[175, 185, 193], [51, 44, 39], [134, 131, 104]]","[[37, 85, 83], [116, 202, 245], [68, 132, 149]]","[[119, 154, 165], [58, 72, 55], [215, 233, 247]]"
orientation,paysage,paysage,paysage,paysage,paysage,paysage,portrait,paysage,portrait,paysage,paysage,paysage,paysage,paysage,paysage
size,Grande,Grande,Petite,Grande,Petite,Moyenne,Moyenne,Grande,Grande,Vignette,Petite,Moyenne,Grande,Grande,Grande
dom,Red,Red,Red,Red,Red,White,White,Red,Blue,Red,Green,Blue,Blue,Green,Blue
